## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

## Preambule <a class="anchor" id="prem"></a>

### Packages <a class = "anchor" id = "package"></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### Functions <a class = "anchor" id="function"></a>

In [20]:
# Function for model architecture
# WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.

# cb = keras.callbacks.EarlyStopping(monitor='loss', patience=20)
def create_model2C(input_shape, nodes_per_layer, dropout, activation):

    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(nodes_per_layer[1], activation=activation),
                        Dense(nodes_per_layer[0], activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model3C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(nodes_per_layer[1], activation=activation, return_sequences=True),
                        LSTM(64, activation=activation),
                        Dense(nodes_per_layer[0], activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model4C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(nodes_per_layer[1], activation=activation, return_sequences=True),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation),
                        Dense(64, activation = 'relu'),
                        Dense(nodes_per_layer[0], activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model


def model001_2C(input_shape, nodes_per_layer, dropout, activation):
    '''
    node = 256, activation = tanh, dropout = 0.3, bs = 64
    '''
    weights_file = "weights_file.h5"

    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=20)
    model = Sequential()
    model.add(LSTM(units=nodes_per_layer[0], activation='sigmoid', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(nodes_per_layer[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.01))
    model.save_weights(weights_file)

    return model

# Function for the rectified RUL
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

# Function for data preprocessing
def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

# LSTM <a class="anchor" id="lstm">  </a>

### FD001  <a class="anchor" id="fd001">  </a>

#### Data loading <a id = "fd001dataload"></a>

In [21]:
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
train = rul_piecewise_fct(train, 130)

(20631, 27) (13096, 26) (100, 1)


#### Model selection <a id = "fd001modelselect"></a>

In [22]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[256, 32], [256, 64]] 

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,3)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh', 'tanh']
batch_size_list = [64, 128]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

3584

In [23]:
ITERATIONS = 1
SEED = 0
rul_piecewise = 130

In [22]:
# %%time
# results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
#                                 'S_score','std_S_score',
#                                 'MSE', 'std_MSE',
#                                 'nodes', 'dropout',
#                                 'activation', 'batch_size'])


# for i in range(ITERATIONS):
#     if ITERATIONS < 10:
#         print('iteration ', i+1)
#     elif ((i+1) % 10 == 0):
#         print('iteration ', i+1)    
#     tf.random.set_seed(SEED)
#     mse = []
#     R2_val = []
#     RMSE = []
#     score_val = []
    
    
# # parameter's sample
#     alpha = 0.3
#     sequence_length = 30
#     epochs = 25
#     nodes_per_layer = random.sample(nodes_list, 1)[0]
#     dropout = random.sample(dropouts, 1)[0]
#     activation = random.sample(activation_functions, 1)[0]
#     batch_size = random.sample(batch_size_list, 1)[0]
#     remaining_sensors = remaining_sensors
#     drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

#     # create model
#     input_shape = (sequence_length, len(remaining_sensors))
#     model = create_model2C(input_shape, nodes_per_layer, dropout, activation)
    
#     # Data prepration
#     X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

#     # create sequences train, test
#     train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
#     label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

#     test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
#                for unit_nr in X_test_interim['Unit'].unique())
    
#     test_array = np.concatenate(list(test_gen)).astype(np.float32)
#     test_rul = rul_piecewise_fct(y_test,rul_piecewise)
#     print(train_array.shape, label_array.shape, test_array.shape)

#     with tf.device('/device:GPU:0'):
#         history = model.fit(train_array, label_array,
#                                 validation_data=(test_array, test_rul),
#                                 epochs=epochs,
#                                 batch_size=batch_size,
#                                 # callbacks=[cb],
#                                 verbose=1)
#         mse.append(history.history['val_loss'][-1])

#         y_hat_val_split = model.predict(test_array)
#         R2_val.append(r2_score(test_rul, y_hat_val_split))
#         RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
#         score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
#     #       append results
#     d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
#          'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
#          'MSE':np.mean(mse), 'std_MSE':np.std(mse),
#          'nodes':str(nodes_per_layer), 'dropout':dropout, 
#          'activation':activation, 'batch_size':batch_size}

# #     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
#     results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(17731, 30, 14) (17731, 1) (100, 30, 14)
Epoch 1/25
278/278 [==============================] - 20s 64ms/step - loss: 2655.6743 - val_loss: 1727.4628
Epoch 2/25
278/278 [==============================] - 18s 64ms/step - loss: 1890.8826 - val_loss: 1679.0699
Epoch 3/25
278/278 [==============================] - 18s 63ms/step - loss: 1886.8331 - val_loss: 1722.1522
Epoch 4/25
278/278 [==============================] - 17s 61ms/step - loss: 1831.5364 - val_loss: 738.6825
Epoch 5/25
278/278 [==============================] - 19s 68ms/step - loss: 444.4911 - val_loss: 232.7926
Epoch 6/25
278/278 [==============================] - 18s 65ms/step - loss: 269.7959 - val_loss: 501.3882
Epoch 7/25
278/278 [==============================] - 17s 62ms/step - loss: 251.8006 - val_loss: 208.9615
Epoch 8/25
278/278 [==============================] - 17s 63ms/step - loss: 231.9857 - val_loss: 234.8961
Epoch 9/25
278/278 [==============================] - 17s 61ms/step - loss: 227.3443 - val_

In [7]:
# results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
3,13.517865,0.0,321.525047,0.0,182.732727,0.0,"[256, 64]",0.2,tanh,128
1,13.850535,0.0,372.648272,0.0,191.837311,0.0,"[256, 128]",0.1,tanh,64
5,13.959899,0.0,352.199345,0.0,194.878784,0.0,"[256, 128]",0.1,tanh,64
8,14.073073,0.0,370.466091,0.0,198.051361,0.0,"[256, 64]",0.2,tanh,128
9,14.126784,0.0,376.765614,0.0,199.566055,0.0,"[256, 128]",0.1,tanh,128
4,14.179949,0.0,409.939201,0.0,201.070984,0.0,"[256, 64]",0.1,tanh,128
7,14.252792,0.0,408.749642,0.0,203.142105,0.0,"[256, 128]",0.1,tanh,64
6,14.374160,0.0,390.931189,0.0,206.616440,0.0,"[256, 64]",0.1,tanh,64
2,14.512681,0.0,445.433419,0.0,210.617889,0.0,"[256, 128]",0.1,tanh,128
0,14.679458,0.0,438.169429,0.0,215.486450,0.0,"[256, 128]",0.2,tanh,128


In [6]:
# %%time
# results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
#                                 'S_score','std_S_score',
#                                 'MSE', 'std_MSE',
#                                 'nodes', 'dropout',
#                                 'activation', 'batch_size'])


# for i in range(ITERATIONS):
#     if ITERATIONS < 10:
#         print('iteration ', i+1)
#     elif ((i+1) % 10 == 0):
#         print('iteration ', i+1)    
#     tf.random.set_seed(SEED)
#     mse = []
#     R2_val = []
#     RMSE = []
#     score_val = []
    
    
# # parameter's sample
#     alpha = 0.3
#     sequence_length = 30
#     epochs = 25
#     nodes_per_layer = random.sample(nodes_list, 1)[0]
#     dropout = random.sample(dropouts, 1)[0]
#     activation = random.sample(activation_functions, 1)[0]
#     batch_size = random.sample(batch_size_list, 1)[0]
#     remaining_sensors = remaining_sensors
#     drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

#     # create model
#     input_shape = (sequence_length, len(remaining_sensors))
#     model = create_model3C(input_shape, nodes_per_layer, dropout, activation)
    
#     # Data prepration
#     X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

#     # create sequences train, test
#     train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
#     label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

#     test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
#                for unit_nr in X_test_interim['Unit'].unique())
    
#     test_array = np.concatenate(list(test_gen)).astype(np.float32)
#     test_rul = rul_piecewise_fct(y_test,rul_piecewise)
#     print(train_array.shape, label_array.shape, test_array.shape)

#     with tf.device('/device:GPU:0'):
#         history = model.fit(train_array, label_array,
#                                 validation_data=(test_array, test_rul),
#                                 epochs=epochs,
#                                 batch_size=batch_size,
#                                 # callbacks=[cb],
#                                 verbose=1)
#         mse.append(history.history['val_loss'][-1])

#         y_hat_val_split = model.predict(test_array)
#         R2_val.append(r2_score(test_rul, y_hat_val_split))
#         RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
#         score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
#     #       append results
#     d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
#          'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
#          'MSE':np.mean(mse), 'std_MSE':np.std(mse),
#          'nodes':str(nodes_per_layer), 'dropout':dropout, 
#          'activation':activation, 'batch_size':batch_size}

# #     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
#     results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(17731, 30, 14) (17731, 1) (100, 30, 14)
Epoch 1/25
278/278 [==============================] - 24s 75ms/step - loss: 2690.4429 - val_loss: 1729.3969
Epoch 2/25
278/278 [==============================] - 18s 63ms/step - loss: 1903.5499 - val_loss: 1675.2544
Epoch 3/25
278/278 [==============================] - 19s 67ms/step - loss: 1901.9313 - val_loss: 1721.7600
Epoch 4/25
278/278 [==============================] - 19s 70ms/step - loss: 1899.9883 - val_loss: 1684.6479
Epoch 5/25
278/278 [==============================] - 19s 67ms/step - loss: 961.8239 - val_loss: 292.1729
Epoch 6/25
278/278 [==============================] - 18s 65ms/step - loss: 283.0903 - val_loss: 509.9312
Epoch 7/25
278/278 [==============================] - 19s 70ms/step - loss: 266.8125 - val_loss: 192.6033
Epoch 8/25
278/278 [==============================] - 20s 72ms/step - loss: 231.3839 - val_loss: 198.8567
Epoch 9/25
278/278 [==============================] - 19s 67ms/step - loss: 226.8311 - val_loss: 188.84

In [8]:
# results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,14.508221,0.0,449.657836,0.0,210.488434,0.0,"[256, 32]",0.2,tanh,64
9,14.536720,0.0,452.679087,0.0,211.316254,0.0,"[256, 64]",0.2,tanh,64
7,14.731667,0.0,481.297785,0.0,217.022034,0.0,"[256, 64]",0.2,tanh,64
1,14.734535,0.0,446.855999,0.0,217.106567,0.0,"[256, 32]",0.1,tanh,128
6,14.813646,0.0,457.662222,0.0,219.444122,0.0,"[256, 32]",0.2,tanh,128
3,15.494307,0.0,543.409197,0.0,240.073517,0.0,"[256, 64]",0.2,tanh,128
8,15.630160,0.0,561.202318,0.0,244.301941,0.0,"[256, 64]",0.1,tanh,128
2,15.709916,0.0,616.340811,0.0,246.801468,0.0,"[256, 64]",0.2,tanh,128
5,15.801034,0.0,575.720540,0.0,249.672653,0.0,"[256, 32]",0.2,tanh,64
4,16.700977,0.0,716.102582,0.0,278.922607,0.0,"[256, 64]",0.1,tanh,128


In [24]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = 64
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model4C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(17731, 30, 14) (17731, 1) (100, 30, 14)
Epoch 1/25
278/278 [==============================] - 24s 74ms/step - loss: 2546.8135 - val_loss: 1746.3311
Epoch 2/25
278/278 [==============================] - 20s 72ms/step - loss: 1896.3131 - val_loss: 1655.9519
Epoch 3/25
278/278 [==============================] - 20s 72ms/step - loss: 1894.5208 - val_loss: 1721.5046
Epoch 4/25
278/278 [==============================] - 20s 71ms/step - loss: 1894.0259 - val_loss: 1691.7137
Epoch 5/25
278/278 [==============================] - 20s 71ms/step - loss: 1891.7537 - val_loss: 1689.7113
Epoch 6/25
278/278 [==============================] - 20s 71ms/step - loss: 1900.5850 - val_loss: 1728.5280
Epoch 7/25
278/278 [==============================] - 20s 72ms/step - loss: 1897.0848 - val_loss: 1728.1847
Epoch 8/25
278/278 [==============================] - 21s 75ms/step - loss: 1896.4448 - val_loss: 1672.9656
Epoch 9/25
278/278 [==============================] - 20s 73ms/step - loss: 1898.2

In [26]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,13.892528,0.0,373.299821,0.0,193.00235,0.0,"[256, 32]",0.1,tanh,64


In [6]:
# %%time
# results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
#                                 'S_score','std_S_score',
#                                 'MSE', 'std_MSE',
#                                 'nodes', 'dropout',
#                                 'activation', 'batch_size'])


# for i in range(ITERATIONS):
#     if ITERATIONS < 10:
#         print('iteration ', i+1)
#     elif ((i+1) % 10 == 0):
#         print('iteration ', i+1)
    
#     tf.random.set_seed(SEED)
#     mse = []
#     R2_val = []
#     RMSE = []
#     score_val = []
    
    
#     # init parameters
#     # Random search
#     # Number of layers = 1
#     # alpha = random.sample(alpha_list, 1)[0]
#     alpha = 0.3
#     # sequence_length = random.sample(sequence_list, 1)[0]
#     sequence_length = 30
#     epochs = 25
# #     epochs = random.sample(epoch_list, 1)[0]
#     nodes_per_layer = random.sample(nodes_list, 1)[0]
#     dropout = random.sample(dropouts, 1)[0]
#     activation = random.sample(activation_functions, 1)[0]
#     batch_size = random.sample(batch_size_list, 1)[0]
#     remaining_sensors = remaining_sensors
#     drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
#     # create model
#     input_shape = (sequence_length, len(remaining_sensors))
#     model = create_model(input_shape, nodes_per_layer, dropout, activation)
    
#     # create train-val split
#     X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
#     test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
#            for unit_nr in X_test_interim['Unit'].unique())
#     test_array = np.concatenate(list(test_gen)).astype(np.float32)
        
#     gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=0)
#     for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
#         train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
#         train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
#         train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)
        
#         val_unit = X_train_interim['Unit'].unique()[val_unit]
#         val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
#         val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)
        
#         # train and evaluate model
#         # model.compile(loss='mean_squared_error', optimizer='adam')
# #         model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration 
#         history = model.fit(train_split_array, train_split_label,
#                             validation_data=(val_split_array, val_split_label),
#                             epochs=epochs,
#                             batch_size=batch_size,
#                             callbacks=[cb],
#                             verbose=0)
#         mse.append(history.history['val_loss'][-1])
        
#         test_rul = rul_piecewise_fct(y_test,rul_piecewise)
#         y_hat_val_split = model.predict(test_array)
#         R2_val.append(r2_score(test_rul, y_hat_val_split))
#         RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
#         score_val.append(compute_s_score(test_rul, y_hat_val_split))
    
#     # append results
#     d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
#          'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
#          'MSE':np.mean(mse), 'std_MSE':np.std(mse),
#          'nodes':str(nodes_per_layer), 'dropout':dropout, 
#          'activation':activation, 'batch_size':batch_size}

# #     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
#     results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)



4/4 [==============================] - 0s 5ms/step
iteration  10
4/4 [==============================] - 0s 2ms/step
iteration  20
4/4 [==============================] - 0s 11ms/step
iteration  30
4/4 [==============================] - 0s 11ms/step
iteration  40
4/4 [==============================] - 0s 5ms/step
iteration  50
4/4 [==============================] - 0s 7ms/step
CPU times: total: 19min 26s
Wall time: 1h 29min 59s


In [7]:
# results.to_csv("results/results_lstm_fd001")
# results.sort_values(by=['S_score'])

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
42,13.964884,0.775777,343.915667,74.926650,190.121938,5.957180,[256],0.3,tanh,64
9,14.151649,0.318230,360.923998,25.834467,200.052210,7.959946,[64],0.3,tanh,64
41,14.062142,0.358300,364.308378,63.508436,203.565669,6.704374,[64],0.2,tanh,64
46,16.368388,3.104848,395.652719,110.278752,349.801570,194.723073,[32],0.4,tanh,64
22,14.488488,0.613202,395.661363,74.371661,200.746068,7.535411,[64],0.2,tanh,64
26,16.245971,3.407711,398.048203,105.688379,357.772507,213.303134,[32],0.3,tanh,64
43,16.533802,4.439844,414.011227,168.087064,396.946386,250.424198,[32],0.1,tanh,64
4,14.539668,1.396782,438.421432,156.077811,188.438594,10.261512,[256],0.4,tanh,64
33,17.224459,3.994632,447.701035,165.744409,398.483220,265.983996,[32],0.3,tanh,64
48,14.722426,0.820359,449.178445,109.075064,214.336370,21.456571,[64],0.1,tanh,64


In [18]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)
    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 30
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model(input_shape, nodes_per_layer, dropout, activation)
    
    # create train-val split
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
           for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
        
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=0)
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)
            
            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)
            
            # train and evaluate model
            # model.compile(loss='mean_squared_error', optimizer='adam')
    #         model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration 
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=0)
            mse.append(history.history['val_loss'][-1])
            
            test_rul = rul_piecewise_fct(y_test,rul_piecewise)
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
        
    #       append results
        d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
            'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
            'MSE':np.mean(mse), 'std_MSE':np.std(mse),
            'nodes':str(nodes_per_layer), 'dropout':dropout, 
            'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
4/4 [==============================] - 0s 19ms/step
CPU times: user 54min 1s, sys: 13min 49s, total: 1h 7min 50s
Wall time: 21min 53s


In [19]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,41.671751,0.274223,25234.254876,3095.685427,1880.872396,10.667098,"[256, 256]",0.1,sigmoid,64


In [13]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,28.589615,10.340965,14485.962438,9981.027034,982.682170,676.837430,"[256, 128]",0.4,tanh,64
1,41.147799,0.066788,19242.143650,797.596495,1872.618693,25.281601,"[256, 32]",0.3,tanh,512
3,41.213384,0.162569,19988.531815,1921.223753,1875.515910,21.979551,"[256, 32]",0.2,tanh,64
2,41.261822,0.085623,20589.415551,991.718543,1873.111938,21.211562,"[256, 64]",0.2,tanh,512


In [39]:
results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### FD002  <a class="anchor" id="fd002">  </a>

In [25]:
# [64]	0.2	tanh	128

def create_model2C(input_shape, nodes_per_layer, dropout, activation):

    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation=activation),
                        Dense(64, activation = 'relu'),
                        Dropout(0.2),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model3C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    bs = 64
    
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(32, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation),
                        Dense(64, activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model4C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(32, activation=activation),
                        Dense(32, activation = 'relu'),
                        Dense(64, activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model


def model001_2C(input_shape, nodes_per_layer, dropout, activation):
    '''
    node = 256, activation = tanh, dropout = 0.3, bs = 64
    '''
    weights_file = "weights_file.h5"

    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=20)
    model = Sequential()
    model.add(LSTM(units=nodes_per_layer[0], activation='sigmoid', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(nodes_per_layer[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.01))
    model.save_weights(weights_file)

    return model

# Function for the rectified RUL
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

# Function for data preprocessing
def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

#### Data loading <a id = "fd002dataload"></a>

In [26]:
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
train = rul_piecewise_fct(train, 130)

(53759, 27) (33991, 26) (259, 1)


Models

In [27]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[64, 64], [64, 32]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,3)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh', 'tanh']
batch_size_list = [128, 64]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

3584

In [22]:
ITERATIONS = 5
SEED = 0
rul_piecewise = 130

#### Model selection <a id = "fd002modelselect"></a>

In [28]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 38
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.2
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model4C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(44139, 38, 14) (44139, 1) (259, 38, 14)
Epoch 1/25
690/690 [==============================] - 26s 31ms/step - loss: 2417.7385 - val_loss: 1983.9803
Epoch 2/25
690/690 [==============================] - 21s 30ms/step - loss: 1939.5264 - val_loss: 1982.6530
Epoch 3/25
690/690 [==============================] - 21s 30ms/step - loss: 1935.9885 - val_loss: 1952.3306
Epoch 4/25
690/690 [==============================] - 20s 29ms/step - loss: 694.8010 - val_loss: 313.0717
Epoch 5/25
690/690 [==============================] - 20s 29ms/step - loss: 356.1960 - val_loss: 277.3177
Epoch 6/25
690/690 [==============================] - 20s 30ms/step - loss: 340.5667 - val_loss: 268.3872
Epoch 7/25
690/690 [==============================] - 26s 38ms/step - loss: 324.3188 - val_loss: 254.2124
Epoch 8/25
690/690 [==============================] - 31s 45ms/step - loss: 325.3454 - val_loss: 220.1299
Epoch 9/25
690/690 [==============================] - 16s 23ms/step - loss: 320.1729 - val_l

In [16]:
# 2 Layers
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,22.877344,0.0,56882.125904,0.0,523.372864,0.0,"[64, 32]",0.2,tanh,128
1,16.491750,0.0,1682.817681,0.0,271.977844,0.0,"[64, 64]",0.2,tanh,128
2,14.304586,0.0,814.459646,0.0,204.621170,0.0,"[64, 32]",0.2,tanh,128
3,14.454804,0.0,818.358242,0.0,208.941376,0.0,"[64, 32]",0.2,tanh,128
4,13.894001,0.0,828.601908,0.0,193.043274,0.0,"[64, 64]",0.2,tanh,128


In [24]:
# 3 layers 
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,14.511870,0.0,801.168680,0.0,210.594376,0.0,"[64, 64]",0.2,tanh,128
1,16.864087,0.0,3375.886937,0.0,284.397430,0.0,"[64, 32]",0.2,tanh,64
2,18.112611,0.0,3691.585384,0.0,328.066650,0.0,"[64, 64]",0.2,tanh,128
3,14.377577,0.0,760.836050,0.0,206.714706,0.0,"[64, 32]",0.2,tanh,128
4,13.911973,0.0,785.093423,0.0,193.542999,0.0,"[64, 32]",0.2,tanh,64


In [29]:
# 4 layers 
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,14.458791,0.0,806.669349,0.0,209.056641,0.0,"[64, 32]",0.2,tanh,64
1,13.522419,0.0,779.026240,0.0,182.855804,0.0,"[64, 32]",0.2,tanh,64
2,15.255057,0.0,818.839181,0.0,232.716782,0.0,"[64, 32]",0.2,tanh,128
3,14.414061,0.0,853.118127,0.0,207.765152,0.0,"[64, 32]",0.2,tanh,64
4,15.128228,0.0,836.288811,0.0,228.863281,0.0,"[64, 64]",0.2,tanh,128


In [9]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)
    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []

    # parameter's sample
    alpha = 0.3
    sequence_length = 30
    epochs = 15
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model(input_shape, nodes_per_layer, dropout, activation)
    
    # create train-val split
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
           for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
        
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=0)
    for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
        train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
        train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
        train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)
        
        val_unit = X_train_interim['Unit'].unique()[val_unit]
        val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
        val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)
        
        # train and evaluate model
        history = model.fit(train_split_array, train_split_label,
                            validation_data=(val_split_array, val_split_label),
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[cb],
                            verbose=0)
        mse.append(history.history['val_loss'][-1])
        
        test_rul = rul_piecewise_fct(y_test,rul_piecewise)
        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
    
    # append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)



9/9 [==============================] - 0s 4ms/step
iteration  10
9/9 [==============================] - 0s 10ms/step
iteration  20
9/9 [==============================] - 0s 12ms/step
iteration  30
9/9 [==============================] - 0s 4ms/step
iteration  40
9/9 [==============================] - 0s 6ms/step
iteration  50
9/9 [==============================] - 0s 15ms/step
CPU times: total: 50min 30s
Wall time: 3h 53min 58s


In [10]:
results.to_csv("results/results_lstm_fd002")
results.sort_values(by=['S_score'])

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
47,14.831516,0.287748,964.328597,84.554237,252.536174,17.892908,[64],0.2,tanh,128
26,14.805402,0.739714,1128.832308,376.061607,254.512899,24.815123,[256],0.2,tanh,128
17,14.963188,0.369970,1142.584571,168.177174,232.057353,18.358876,[128],0.2,tanh,64
27,14.293991,0.894635,1151.887167,383.437056,245.601725,27.634230,[64],0.1,tanh,64
10,14.882330,0.451319,1181.642982,438.739383,266.680257,13.604762,[128],0.3,tanh,128
36,14.490891,0.082729,1214.240597,183.147265,240.145737,24.178127,[32],0.2,tanh,64
29,15.302502,1.085625,1325.333676,648.526013,265.947968,21.246164,[256],0.2,tanh,128
33,15.165437,1.208680,1361.339447,754.663156,257.504644,36.099646,[256],0.3,tanh,128
8,17.410164,2.663108,1754.234416,520.296523,359.644979,143.454438,[64],0.3,tanh,256
42,17.695608,1.817548,3034.631001,348.437438,299.014175,68.932800,[32],0.4,tanh,128


## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### FD003  <a class="anchor" id="fd003">  </a>

#### Data loading <a id = "fd003dataload"></a>

In [20]:
def create_model2C(input_shape, nodes_per_layer, dropout, activation):

    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation=activation),
                        Dense(nodes_per_layer[0], activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model3C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    # bs = 64
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation),
                        Dense(32, activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model4C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(nodes_per_layer[0], activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation, return_sequences=True),
                        LSTM(64, activation=activation),
                        Dense(32, activation = 'relu'),
                        Dense(64, activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model


def model001_2C(input_shape, nodes_per_layer, dropout, activation):
    '''
    node = 256, activation = tanh, dropout = 0.3, bs = 64
    '''
    weights_file = "weights_file.h5"

    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=20)
    model = Sequential()
    model.add(LSTM(units=nodes_per_layer[0], activation='sigmoid', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(nodes_per_layer[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.01))
    model.save_weights(weights_file)

    return model

# Function for the rectified RUL
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

# Function for data preprocessing
def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

In [21]:
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
train = rul_piecewise_fct(train, 125)

(24720, 27) (16596, 26) (100, 1)


In [22]:
# [32]	0.4	tanh	64
# 0.3 125 40

# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32, 64], [32, 32]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,3)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh', 'tanh']
batch_size_list = [64]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options
ITERATIONS = 10
SEED = 0
rul_piecewise = 125

In [12]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model2C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(20820, 40, 14) (20820, 1) (100, 40, 14)
Epoch 1/20
326/326 [==============================] - 5s 11ms/step - loss: 4320.3857 - val_loss: 1603.5320
Epoch 2/20
326/326 [==============================] - 3s 10ms/step - loss: 1935.7366 - val_loss: 1651.4017
Epoch 3/20
326/326 [==============================] - 3s 10ms/step - loss: 1958.1536 - val_loss: 1681.0681
Epoch 4/20
326/326 [==============================] - 3s 10ms/step - loss: 1934.7556 - val_loss: 1674.3405
Epoch 5/20
326/326 [==============================] - 3s 10ms/step - loss: 1962.9840 - val_loss: 1644.9933
Epoch 6/20
326/326 [==============================] - 3s 10ms/step - loss: 1949.6104 - val_loss: 1669.2524
Epoch 7/20
326/326 [==============================] - 3s 10ms/step - loss: 1841.5752 - val_loss: 522.1207
Epoch 8/20
326/326 [==============================] - 3s 10ms/step - loss: 584.3326 - val_loss: 282.4592
Epoch 9/20
326/326 [==============================] - 3s 10ms/step - loss: 457.8391 - val_loss: 313.3980
E

In [14]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
4,12.721274,0.0,252.834971,0.0,161.830795,0.0,"[32, 64]",0.3,tanh,64
7,13.098344,0.0,310.562840,0.0,171.566635,0.0,"[32, 64]",0.3,tanh,64
3,13.110292,0.0,288.555443,0.0,171.879791,0.0,"[32, 32]",0.3,tanh,64
5,13.186811,0.0,315.361035,0.0,173.891953,0.0,"[32, 32]",0.3,tanh,64
8,13.202157,0.0,309.432383,0.0,174.296921,0.0,"[32, 32]",0.3,tanh,64
2,13.372850,0.0,310.274731,0.0,178.833084,0.0,"[32, 32]",0.3,tanh,64
1,13.414212,0.0,571.424493,0.0,179.941071,0.0,"[32, 32]",0.3,tanh,64
6,13.494769,0.0,392.801972,0.0,182.108765,0.0,"[32, 64]",0.3,tanh,64
0,13.719877,0.0,420.625730,0.0,188.235016,0.0,"[32, 32]",0.3,tanh,64
9,13.931069,0.0,468.297174,0.0,194.074692,0.0,"[32, 64]",0.3,tanh,64


In [18]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model3C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(20820, 40, 14) (20820, 1) (100, 40, 14)
Epoch 1/20
326/326 [==============================] - 9s 20ms/step - loss: 5428.1226 - val_loss: 1622.2159
Epoch 2/20
326/326 [==============================] - 5s 16ms/step - loss: 2183.5127 - val_loss: 1622.1790
Epoch 3/20
326/326 [==============================] - 5s 16ms/step - loss: 2120.4612 - val_loss: 1625.1847
Epoch 4/20
326/326 [==============================] - 5s 16ms/step - loss: 2127.3665 - val_loss: 1628.2081
Epoch 5/20
326/326 [==============================] - 5s 16ms/step - loss: 2137.5244 - val_loss: 1554.7216
Epoch 6/20
326/326 [==============================] - 5s 16ms/step - loss: 2089.0591 - val_loss: 1544.2635
Epoch 7/20
326/326 [==============================] - 5s 16ms/step - loss: 938.3187 - val_loss: 363.0216
Epoch 8/20
326/326 [==============================] - 6s 17ms/step - loss: 622.8356 - val_loss: 263.5007
Epoch 9/20
326/326 [==============================] - 6s 17ms/step - loss: 548.8660 - val_loss: 245.5668
Ep

In [19]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
5,12.032895,0.0,235.421067,0.0,144.790588,0.0,"[32, 64]",0.3,tanh,64
9,12.472302,0.0,279.885393,0.0,155.558319,0.0,"[32, 64]",0.3,tanh,64
0,12.485813,0.0,299.417289,0.0,155.895523,0.0,"[32, 32]",0.3,tanh,64
2,12.778531,0.0,307.900036,0.0,163.290878,0.0,"[32, 64]",0.3,tanh,64
6,12.952646,0.0,397.209433,0.0,167.771057,0.0,"[32, 64]",0.3,tanh,64
1,13.105003,0.0,353.504548,0.0,171.741119,0.0,"[32, 64]",0.3,tanh,64
7,13.412764,0.0,254.972358,0.0,179.902252,0.0,"[32, 32]",0.3,tanh,64
8,13.494967,0.0,426.002984,0.0,182.114136,0.0,"[32, 32]",0.3,tanh,64
3,13.518773,0.0,518.512123,0.0,182.757233,0.0,"[32, 32]",0.3,tanh,64
4,13.710979,0.0,333.747896,0.0,187.990936,0.0,"[32, 32]",0.3,tanh,64


In [23]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model4C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(20820, 40, 14) (20820, 1) (100, 40, 14)
Epoch 1/20
326/326 [==============================] - 12s 26ms/step - loss: 3145.7966 - val_loss: 1656.7677
Epoch 2/20
326/326 [==============================] - 7s 23ms/step - loss: 1897.3990 - val_loss: 1616.4978
Epoch 3/20
326/326 [==============================] - 7s 23ms/step - loss: 1899.6655 - val_loss: 1700.0643
Epoch 4/20
326/326 [==============================] - 7s 23ms/step - loss: 1874.0341 - val_loss: 1661.0127
Epoch 5/20
326/326 [==============================] - 7s 23ms/step - loss: 1883.7297 - val_loss: 1693.8285
Epoch 6/20
326/326 [==============================] - 8s 25ms/step - loss: 1888.5500 - val_loss: 1652.5763
Epoch 7/20
326/326 [==============================] - 8s 24ms/step - loss: 1519.4968 - val_loss: 558.3528
Epoch 8/20
326/326 [==============================] - 8s 24ms/step - loss: 514.8961 - val_loss: 356.6377
Epoch 9/20
326/326 [==============================] - 8s 25ms/step - loss: 398.9040 - val_loss: 340.5733


In [24]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
8,11.887657,0.0,246.921235,0.0,141.316360,0.0,"[32, 32]",0.3,tanh,64
2,12.065069,0.0,209.999264,0.0,145.565903,0.0,"[32, 64]",0.3,tanh,64
7,12.110960,0.0,205.332510,0.0,146.675339,0.0,"[32, 32]",0.3,tanh,64
6,12.142549,0.0,257.206619,0.0,147.441483,0.0,"[32, 32]",0.3,tanh,64
3,12.147590,0.0,264.618308,0.0,147.563919,0.0,"[32, 32]",0.3,tanh,64
4,12.171956,0.0,273.558709,0.0,148.156540,0.0,"[32, 32]",0.3,tanh,64
5,12.483033,0.0,308.818673,0.0,155.826126,0.0,"[32, 32]",0.3,tanh,64
1,12.707685,0.0,230.847180,0.0,161.485260,0.0,"[32, 32]",0.3,tanh,64
9,12.885747,0.0,307.268455,0.0,166.042465,0.0,"[32, 32]",0.3,tanh,64
0,13.436948,0.0,267.266505,0.0,180.551605,0.0,"[32, 64]",0.3,tanh,64


#### Model selection <a id = "fd003modelselect"></a>

In [12]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)
    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []

    # parameter's sample
    alpha = 0.3
    sequence_length = 40
    epochs = 15
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model(input_shape, nodes_per_layer, dropout, activation)
    
    # create train-val split
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
           for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
        
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=0)
    for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
        train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
        train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
        train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)
        
        val_unit = X_train_interim['Unit'].unique()[val_unit]
        val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
        val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)
        
        # train and evaluate model
        history = model.fit(train_split_array, train_split_label,
                            validation_data=(val_split_array, val_split_label),
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[cb],
                            verbose=0)
        mse.append(history.history['val_loss'][-1])
        
        test_rul = rul_piecewise_fct(y_test,rul_piecewise)
        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
    
    # append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

4/4 [==============================] - 0s 4ms/step
iteration  10
4/4 [==============================] - 0s 7ms/step
iteration  20
4/4 [==============================] - 0s 5ms/step
iteration  30
4/4 [==============================] - 0s 3ms/step
iteration  40
4/4 [==============================] - 0s 13ms/step
iteration  50
4/4 [==============================] - 0s 9ms/step
CPU times: total: 25min 11s
Wall time: 2h 28min 25s


In [13]:
results.to_csv("results/results_lstm_fd003")
results.sort_values(by=['S_score'])

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
9,17.497979,3.562215,825.774870,401.682215,400.173126,298.613398,[32],0.4,tanh,64
41,16.745512,0.604739,965.182981,367.017037,193.470510,66.504243,[256],0.3,tanh,64
14,16.503820,2.025543,1006.692206,529.569084,190.939529,69.731883,[128],0.3,tanh,64
49,17.605326,0.619697,1112.880867,39.172785,326.713832,226.434678,[128],0.1,tanh,64
19,18.371744,1.790792,1142.275679,218.889242,364.033732,287.240205,[32],0.1,tanh,64
35,18.700393,2.609535,1161.005507,636.561197,370.646434,205.968875,[256],0.2,tanh,128
16,17.011934,2.814851,1263.985011,916.279482,215.209880,101.440674,[64],0.2,tanh,64
45,17.625022,1.263987,1487.148969,48.503773,228.362208,120.898029,[64],0.2,tanh,64
13,19.126311,2.110756,1665.596473,449.880854,232.322617,91.951862,[64],0.1,tanh,64
31,29.325745,9.705228,2807.163994,2002.520789,1359.117264,1006.017335,[32],0.3,tanh,128


## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### FD004  <a class="anchor" id="fd004">  </a>

In [34]:
# [64]	0.2	tanh	128
# 0.2 125 39

def create_model2C(input_shape, nodes_per_layer, dropout, activation):

    # weights_file = 'weights_file.h5'
    
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation='tanh'),
                        Dense(256, activation = 'relu'),
                        Dropout(0.2),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model3C(input_shape, nodes_per_layer, dropout, activation):
    
    # weights_file = 'weights_file.h5'
    # bs = 64
    
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation='tanh', return_sequences=True),
                        LSTM(32, activation='tanh'),
                        Dense(256, activation = 'relu'),
                        Dropout(0.2),
                        Dense(1)
                        ])
    # model.save_weights(weights_file)
    
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

def create_model4C(input_shape, nodes_per_layer, dropout, activation):
        
    model = Sequential([LSTM(64, activation='tanh', return_sequences=True),
                        # Dropout(dropout),
                        LSTM(64, activation='tanh', return_sequences=True),
                        LSTM(32, activation='tanh', return_sequences=True),
                        LSTM(nodes_per_layer[1], activation='tanh'),
                        Dense(32, activation = 'relu'),
                        Dense(256, activation = 'relu'),
                        Dropout(dropout),
                        Dense(1)
                        ])
        
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))

    return model

# Function for the rectified RUL
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

# Function for data preprocessing
def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

#### Data loading <a id = "fd004dataload"></a>

In [35]:
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
train = rul_piecewise_fct(train, 125)

(61249, 27) (41214, 26) (248, 1)


In [36]:
# [256]	0.3	tanh	64
# 0.2 125 39

# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [ [256, 64], [256, 32]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,3)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh', 'tanh']
batch_size_list = [64, 128]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options
ITERATIONS = 1
SEED = 0
rul_piecewise = 125

In [37]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.2
    sequence_length = 39
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model4C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(51787, 39, 14) (51787, 1) (248, 39, 14)
Epoch 1/20
405/405 [==============================] - 20s 40ms/step - loss: 2400.6450 - val_loss: 1934.7534
Epoch 2/20
405/405 [==============================] - 14s 35ms/step - loss: 1778.7178 - val_loss: 1922.0099
Epoch 3/20
405/405 [==============================] - 15s 37ms/step - loss: 1516.0817 - val_loss: 430.3576
Epoch 4/20
405/405 [==============================] - 16s 38ms/step - loss: 343.0865 - val_loss: 265.8938
Epoch 5/20
405/405 [==============================] - 16s 39ms/step - loss: 286.6220 - val_loss: 284.0049
Epoch 6/20
405/405 [==============================] - 16s 39ms/step - loss: 267.3003 - val_loss: 240.4367
Epoch 7/20
405/405 [==============================] - 16s 39ms/step - loss: 255.1975 - val_loss: 246.1687
Epoch 8/20
405/405 [==============================] - 16s 39ms/step - loss: 239.7551 - val_loss: 228.8415
Epoch 9/20
405/405 [==============================] - 16s 39ms/step - loss: 229.3929 - val_lo

In [38]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,15.031585,0.0,1682.405336,0.0,225.948593,0.0,"[256, 64]",0.3,tanh,128


In [12]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
7,14.108670,0.0,1187.232245,0.0,199.054581,0.0,"[256, 64]",0.3,tanh,128
3,14.506189,0.0,918.364225,0.0,210.429535,0.0,"[256, 128]",0.3,tanh,64
2,14.539137,0.0,999.440788,0.0,211.386520,0.0,"[256, 32]",0.3,tanh,64
0,34.012004,0.0,266389.055452,0.0,1156.816528,0.0,"[256, 128]",0.3,tanh,128
6,34.154820,0.0,287564.466358,0.0,1166.551636,0.0,"[256, 64]",0.3,tanh,64
4,34.262780,0.0,285836.886247,0.0,1173.938110,0.0,"[256, 128]",0.3,tanh,64
5,35.243336,0.0,419212.856331,0.0,1242.092773,0.0,"[256, 64]",0.3,tanh,64
8,35.290780,0.0,423244.192450,0.0,1245.439209,0.0,"[256, 32]",0.3,tanh,64
1,36.313713,0.0,544881.207506,0.0,1318.685791,0.0,"[256, 64]",0.3,tanh,128


In [8]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.2
    sequence_length = 39
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model3C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(51787, 39, 14) (51787, 1) (248, 39, 14)
Epoch 1/20
810/810 [==============================] - 37s 43ms/step - loss: 2131.9399 - val_loss: 2329.2988
Epoch 2/20
810/810 [==============================] - 38s 47ms/step - loss: 1761.0864 - val_loss: 2318.7090
Epoch 3/20
810/810 [==============================] - 36s 45ms/step - loss: 509.3386 - val_loss: 340.8644
Epoch 4/20
810/810 [==============================] - 36s 44ms/step - loss: 303.0032 - val_loss: 298.2368
Epoch 5/20
810/810 [==============================] - 38s 47ms/step - loss: 265.1307 - val_loss: 249.6993
Epoch 6/20
810/810 [==============================] - 38s 47ms/step - loss: 250.5625 - val_loss: 244.8042
Epoch 7/20
810/810 [==============================] - 36s 45ms/step - loss: 236.3432 - val_loss: 241.0859
Epoch 8/20
810/810 [==============================] - 35s 43ms/step - loss: 223.8640 - val_loss: 202.0907
Epoch 9/20
810/810 [==============================] - 35s 43ms/step - loss: 214.2158 - val_loss: 198.5914
E

In [10]:
results.sort_values('RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
1,12.992329,0.0,751.370703,0.0,168.800629,0.0,"[256, 32]",0.3,tanh,64
5,13.233821,0.0,780.216294,0.0,175.134003,0.0,"[256, 64]",0.3,tanh,64
4,13.410653,0.0,874.103563,0.0,179.845642,0.0,"[256, 32]",0.3,tanh,64
0,13.530306,0.0,945.027037,0.0,183.069214,0.0,"[256, 64]",0.3,tanh,64
9,13.838789,0.0,985.140372,0.0,191.512085,0.0,"[256, 32]",0.3,tanh,128
6,14.569964,0.0,1051.923955,0.0,212.283844,0.0,"[256, 64]",0.3,tanh,64
7,26.566606,0.0,130894.519913,0.0,705.784546,0.0,"[256, 64]",0.3,tanh,128
2,27.218556,0.0,126892.621834,0.0,740.849792,0.0,"[256, 32]",0.3,tanh,64
8,27.502641,0.0,41285.524330,0.0,756.395264,0.0,"[256, 64]",0.3,tanh,64
3,29.461546,0.0,85932.843265,0.0,867.982605,0.0,"[256, 64]",0.3,tanh,64


In [17]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    alpha = 0.2
    sequence_length = 39
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = 0.3
    activation = 'tanh'
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model4C(input_shape, nodes_per_layer, dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(51787, 39, 14) (51787, 1) (248, 39, 14)
Epoch 1/20
405/405 [==============================] - 17s 35ms/step - loss: 2927.1733 - val_loss: 1911.2460
Epoch 2/20
405/405 [==============================] - 13s 32ms/step - loss: 1870.3485 - val_loss: 1901.4124
Epoch 3/20
405/405 [==============================] - 18s 44ms/step - loss: 1855.0073 - val_loss: 1250.2932
Epoch 4/20
405/405 [==============================] - 19s 48ms/step - loss: 570.4372 - val_loss: 331.8809
Epoch 5/20
405/405 [==============================] - 19s 46ms/step - loss: 417.6230 - val_loss: 341.8280
Epoch 6/20
405/405 [==============================] - 19s 47ms/step - loss: 399.6472 - val_loss: 318.0155
Epoch 7/20
405/405 [==============================] - 18s 44ms/step - loss: 387.8885 - val_loss: 326.5475
Epoch 8/20
405/405 [==============================] - 16s 40ms/step - loss: 373.5564 - val_loss: 304.5685
Epoch 9/20
405/405 [==============================] - 16s 41ms/step - loss: 355.7402 - val_l

In [18]:
results.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
2,13.671511,0.0,905.976041,0.0,186.910202,0.0,"[256, 32]",0.3,tanh,64
1,13.757116,0.0,893.839576,0.0,189.258224,0.0,"[256, 64]",0.3,tanh,64
3,14.214624,0.0,1003.773496,0.0,202.055542,0.0,"[256, 32]",0.3,tanh,128
4,14.982881,0.0,1314.572553,0.0,224.486740,0.0,"[256, 64]",0.3,tanh,64
0,15.871048,0.0,13994.589940,0.0,251.890152,0.0,"[256, 32]",0.3,tanh,128


#### Model selection <a id = "fd004modelselect"></a>

In [15]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)
    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []

    # parameter's sample
    alpha = 0.3
    sequence_length = 30
    epochs = 15
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_model(input_shape, nodes_per_layer, dropout, activation)
    
    # create train-val split
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
           for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
        
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=0)
    for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
        train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
        train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
        train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)
        
        val_unit = X_train_interim['Unit'].unique()[val_unit]
        val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
        val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)
        
        # train and evaluate model
        history = model.fit(train_split_array, train_split_label,
                            validation_data=(val_split_array, val_split_label),
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[cb],
                            verbose=0)
        mse.append(history.history['val_loss'][-1])
        
        test_rul = rul_piecewise_fct(y_test,rul_piecewise)
        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
    
    # append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

8/8 [==============================] - 0s 8ms/step
iteration  10
8/8 [==============================] - 0s 9ms/step
iteration  20
8/8 [==============================] - 0s 7ms/step
iteration  30
8/8 [==============================] - 0s 16ms/step
iteration  40
8/8 [==============================] - 0s 5ms/step
iteration  50
8/8 [==============================] - 0s 16ms/step
CPU times: total: 1h 9min 55s
Wall time: 14h 51min 56s


In [16]:
results.to_csv("results/results_lstm_fd004")
results.sort_values(by=['S_score'])

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
37,16.014446,1.122225,1648.018057,430.508037,234.990128,49.712271,[256],0.3,tanh,64
29,16.719837,0.857054,1658.167758,273.375191,239.616842,37.855458,[256],0.3,tanh,128
7,16.770742,0.787315,1942.513307,398.675394,262.900579,52.261578,[256],0.1,tanh,64
13,17.736544,1.348821,2571.548906,380.349717,269.373947,73.964361,[32],0.1,tanh,64
38,18.110949,1.982117,2752.945688,1603.314743,276.890533,58.884608,[256],0.2,tanh,256
44,18.959587,0.888993,2957.138854,425.307524,253.749090,59.223473,[64],0.2,tanh,128
22,17.774564,0.710162,2999.142898,298.172653,244.539729,35.950421,[256],0.4,tanh,128
28,18.372242,1.679014,3110.906198,920.607156,293.228841,68.834558,[64],0.2,tanh,128
0,18.303144,2.009490,3376.373134,2269.266565,242.369481,61.523606,[128],0.3,tanh,128
30,19.513405,2.928476,3791.624621,2108.094207,326.621857,123.281974,[256],0.1,tanh,256
